In [1]:
import pandas as pd
import requests
from gazpacho import Soup
import numpy as np

# Extract

In [2]:
url = 'https://www.imdb.com/search/title/?groups=top_100&sort=user_rating,desc'

In [3]:
request = requests.get(url)

In [4]:
imdb = Soup(request.text)

In [5]:
movies = []

In [6]:
# get movie name
for movie in imdb.find('h3',{'class':'lister-item-header'}):
    movies.append(movie.strip())

In [7]:
rank = []

In [8]:
# get ranking
for i in movies:
    rank.append(i.split('.')[0])

In [9]:
runtime = []

In [10]:
# get time
for time in imdb.find('span',{'class':'runtime'}):
    runtime.append(time.strip())

In [11]:
types = []

In [12]:
# get type of movie
for t in imdb.find('span',{'class':'genre'}):
    types.append(t.strip())

In [13]:
description= []

In [14]:
for idx, i in enumerate(imdb.find('p',{'class':'text-muted'})):
    if (idx % 2) ==1 :
        description.append(i.strip())

In [15]:
votes = []
gross = []

In [16]:
for i in imdb.find('p',{'class':'sort-num_votes-visible'}):
#     votes.append(t.strip())
    num_votes = i.strip()
    vote = num_votes.split(' | ')
    votes.append(vote[0])
    gross.append(vote[1])

In [17]:
votes[:5]

['Votes: 2,747,979',
 'Votes: 1,911,287',
 'Votes: 62,493',
 'Votes: 2,720,714',
 'Votes: 1,386,096']

In [18]:
gross[:5]

['Gross: $28.34M',
 'Gross: $134.97M',
 'Top 250: #15',
 'Gross: $534.86M',
 'Gross: $96.90M']

# Transform 

In [19]:
dict = {'rank':rank,'movie': movies, 'runtime(min)': runtime,'types':types,'votes':votes,'gross(M)':gross,'description':description}

In [20]:
# create dataframe
df = pd.DataFrame(dict)

In [21]:
df.head()

,rank,movie,runtime(min),types,votes,gross(M),description
0,1,1. The Shawshank Redemption (1994),142 min,Drama,"Votes: 2,747,979",Gross: $28.34M,"Over the course of several years, two convicts..."
1,2,2. The Godfather (1972),175 min,"Crime, Drama","Votes: 1,911,287",Gross: $134.97M,"Don Vito Corleone, head of a mafia family, dec..."
2,3,3. Spider-Man: Across the Spider-Verse (2023),140 min,"Animation, Action, Adventure","Votes: 62,493",Top 250: #15,"Miles Morales catapults across the Multiverse,..."
3,4,4. The Dark Knight (2008),152 min,"Action, Crime, Drama","Votes: 2,720,714",Gross: $534.86M,When the menace known as the Joker wreaks havo...
4,5,5. Schindler's List (1993),195 min,"Biography, Drama, History","Votes: 1,386,096",Gross: $96.90M,"In German-occupied Poland during World War II,..."


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   rank          50 non-null     object
 1   movie         50 non-null     object
 2   runtime(min)  50 non-null     object
 3   types         50 non-null     object
 4   votes         50 non-null     object
 5   gross(M)      50 non-null     object
 6   description   50 non-null     object
dtypes: object(7)
memory usage: 2.9+ KB


In [23]:
# Change rank column type
df["rank"] = df["rank"].astype('int')

In [24]:
# add year column
df['year'] = df['movie'].map(lambda x: x.split(' ')[-1])

In [25]:
# clean and change year column type
df["year"] = df["year"].map(lambda x: x.lstrip('(').rstrip(')')).astype('int')

In [26]:
# clean and change runtime column type
df['runtime(min)'] = df['runtime(min)'].map(lambda x: x.rstrip('min')).astype('int')

In [27]:
# clean and change votes column type
df["votes"] = df["votes"].map(lambda x: x.lstrip('Votes: ')).str.replace(',','').astype('int')

In [28]:
# clean and change gross column type
df["gross(M)"] = df["gross(M)"].map(lambda x: x.lstrip('Gross: $').rstrip('M'))

In [29]:
data_null = df['gross(M)'].str.contains('Top')

In [ ]:
df['gross(M)'][data_null] = 0

In [31]:
df['gross(M)'] = df['gross(M)'].astype('float')

In [32]:
# clean movie column
df["movie"] = df["movie"].str.replace('[^a-zA-Z ]','',regex=True)

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   rank          50 non-null     int32  
 1   movie         50 non-null     object 
 2   runtime(min)  50 non-null     int32  
 3   types         50 non-null     object 
 4   votes         50 non-null     int32  
 5   gross(M)      50 non-null     float64
 6   description   50 non-null     object 
 7   year          50 non-null     int32  
dtypes: float64(1), int32(4), object(3)
memory usage: 2.5+ KB


In [34]:
df.head()

,rank,movie,runtime(min),types,votes,gross(M),description,year
0,1,The Shawshank Redemption,142,Drama,2747979,28.34,"Over the course of several years, two convicts...",1994
1,2,The Godfather,175,"Crime, Drama",1911287,134.97,"Don Vito Corleone, head of a mafia family, dec...",1972
2,3,SpiderMan Across the SpiderVerse,140,"Animation, Action, Adventure",62493,0.00,"Miles Morales catapults across the Multiverse,...",2023
3,4,The Dark Knight,152,"Action, Crime, Drama",2720714,534.86,When the menace known as the Joker wreaks havo...,2008
4,5,Schindlers List,195,"Biography, Drama, History",1386096,96.90,"In German-occupied Poland during World War II,...",1993


# Load

In [35]:
import mysql.connector

In [36]:
conn = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "",
    database = "movie_ranking"
)

In [37]:
cursor = conn.cursor()

In [38]:
query = """CREATE TABLE IF NOT EXISTS imdb_ranking (
            id INT PRIMARY KEY AUTO_INCREMENT, 
            rank INT, 
            movie VARCHAR(225), 
            runtime_min INT, 
            types VARCHAR(225),
            gross_million FlOAT, 
            description VARCHAR(225), 
            year INT)"""

In [39]:
cursor.execute(query)


In [40]:
query = 'INSERT INTO imdb_ranking (rank, movie, runtime_min, types, gross_million, description, year) VALUES (%s,%s,%s,%s,%s,%s,%s)'

In [41]:
for index, row in df.iterrows():
        rank = row['rank']
        movie = row['movie']
        runtime_min = row['runtime(min)']
        types = row['types']
        gross_million = row['gross(M)']
        description = row['description']
        year = row['year']
        cursor.execute(query, (rank, movie, runtime_min, types, gross_million, description, year))

In [42]:
conn.commit()
cursor.close()
conn.close()

In [43]:
#save to csv
df.to_csv('imdb_movie.csv')